# Data Access and Cleaning

The purpose of this notebook is to download the subsurface data from the Washington State Department of Natural Resources (DNR) [Geologic Information Portal](https://www.dnr.wa.gov/geologyportal), available for [download](https://www.dnr.wa.gov/programs-and-services/geology/publications-and-data/gis-data-and-databases). 

This data is then filtered to our desired area of interest, clipped to the extent of the study area, and saved as a GeoPackage file.

Finally, the data is loaded for exploratory data analysis.

#### Import libraries

In [1]:
import requests
import zipfile
import os

____________________

#### Download data

In [2]:
# Define the URL and save path for the zip file
url = 'https://fortress.wa.gov/dnr/geologydata/publications/data_download/ger_portal_subsurface_database.zip'
save_path = '../data/temp/ger_portal_subsurface_database.zip'
extract_path = '../data/temp/extracted_files/'

In [3]:
# Ensure the target folders exist
os.makedirs(os.path.dirname(save_path), exist_ok=True)
os.makedirs(extract_path, exist_ok=True)

In [4]:
try:
    with requests.get(url, stream=True) as response:
        response.raise_for_status()
        with open(save_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):  # Download in chunks
                file.write(chunk)
    print(f"Downloaded file saved to {save_path}")
except requests.exceptions.RequestException as e:
    print(f"Failed to download the file. Error: {e}")
    exit(1)

Downloaded file saved to ../data/temp/ger_portal_subsurface_database.zip


In [6]:
# Check if the file is a valid ZIP file
if zipfile.is_zipfile(save_path):
    try:
        with zipfile.ZipFile(save_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
        print(f"Files extracted to {extract_path}")
    except zipfile.BadZipFile as e:
        print(f"Error: The file at {save_path} is a bad ZIP file. {str(e)}")
else:
    print(f"Error: The file at {save_path} is not recognized as a valid ZIP file.")

Files extracted to ../data/temp/extracted_files/


____________________

#### Filter and clip data

____________________

#### Exploratory Data Analysis